In [ ]:
#This program counts cell activity per session, and combines it with PC identification

In [ ]:
#import libraries
import pandas as pd
import glob
import os
import numpy as np
from scipy import signal
import math

#define path names and read in files
path = r'/Users/insert_folder_path_here' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))  
event1 = pd.read_csv(r'/Users/file_path_for_session_1_interpolated_data.csv')
event2 = pd.read_csv(r'/Users/file_path_for_session_2_interpolated_data.csv')
event3 = pd.read_csv(r'/Users/file_path_for_session_3_interpolated_data.csv')
event4 = pd.read_csv(r'/Users/file_path_for_session_4_interpolated_data.csv')
event5 = pd.read_csv(r'/Users/file_path_for_session_5_interpolated_data.csv')
event6 = pd.read_csv(r'/Users/file_path_for_session_6_interpolated_data.csv')
testName = 'Insert_Mouse_ID_here'

output_dir = path + r'/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [ ]:
#To print every row, if needed
#pd.set_option('display.max_rows', None)

#define number of cells detected (just set to number greater than number of cells)
N_cells=700
#defines whether the data is pruned to remove the times the mouse doesn't move (1=pruning on, 0=pruning off)
pruning =1

#defines the threshold for counting events
Spike_threshold=5

#defines the threshold velocity for pruning
#if the velocity os below this, it is pruned when pruning = True
speed_velocity = 1
Threshold_velocity=0.01
#defines the threshold velocity for pruning 
#If using n% of average speed,
average_speed = 0
speed_fraction=0.1

#define timestep size
Delta_time=0.049962


In [ ]:
#define velocity pruning function 
#prunes data with velocity less than threshold velocity from the input data
#prunes spikes with an amplitude less than spike threshold
if speed_velocity==1:
    def prune (df):
        #prune non-moving times
        pruned_velocity=df.drop(df[df.Speed < Threshold_velocity].index).reset_index(drop=True)
        #set spikes with size less than a threshold to zero  
        cell_data_only=pruned_velocity.iloc[:,5:N_cells+5] #get cell data only
        behavioural_data_only=pruned_velocity.iloc[:,0:5] #get behavioural data only
        cell_data_only=cell_data_only.mask(cell_data_only<Spike_threshold, 0) #mask cell data
        pruned_data=pd.concat([behavioural_data_only, cell_data_only], axis = 1) #recombine
        
        return(pruned_data)

In [ ]:
#define velocity pruning function by using average speed 
#prunes data with velocity less than threshold velocity from the input data
#prunes spikes with an amplitude less than spike threshold
if average_speed==1:
    def prune (df):
        average_speed= df['Speed'].mean()
        thresh_speed=average_speed*speed_fraction
        pruned_velocity=df.drop(df[df.Speed < thresh_speed].index).reset_index(drop=True)
        #set spikes with size less than a threshold to zero  
        cell_data_only=pruned_velocity.iloc[:,5:N_cells+5] #get cell data only
        behavioural_data_only=pruned_velocity.iloc[:,0:5] #get behavioural data only
        cell_data_only=cell_data_only.mask(cell_data_only<Spike_threshold, 0) #mask cell data
        pruned_data=pd.concat([behavioural_data_only, cell_data_only], axis = 1) #recombine
        
        return(pruned_data)

## Activation check starts here

In [ ]:
#clean column names
event1.columns = [c.replace(' ', '') for c in event1.columns]
event2.columns = [c.replace(' ', '') for c in event2.columns]
event3.columns = [c.replace(' ', '') for c in event3.columns]
event4.columns = [c.replace(' ', '') for c in event4.columns]
event5.columns = [c.replace(' ', '') for c in event5.columns]
event6.columns = [c.replace(' ', '') for c in event6.columns]

In [ ]:
#read in the combined Ca imaging and behavioural data for one session, delete intermediate data, rename speed column
event1=event1.drop(['Unnamed:0','Interpolatedspeed', 'x-velocity','y-velocity'], axis=1)
event1=event1.rename({'Calculatedspeed': 'Speed'}, axis=1)
event2=event2.drop(['Unnamed:0','Interpolatedspeed', 'x-velocity','y-velocity'], axis=1)
event2=event2.rename({'Calculatedspeed': 'Speed'}, axis=1)
event3=event3.drop(['Unnamed:0','Interpolatedspeed', 'x-velocity','y-velocity'], axis=1)
event3=event3.rename({'Calculatedspeed': 'Speed'}, axis=1)
event4=event4.drop(['Unnamed:0','Interpolatedspeed', 'x-velocity','y-velocity'], axis=1)
event4=event4.rename({'Calculatedspeed': 'Speed'}, axis=1)
event5=event5.drop(['Unnamed:0','Interpolatedspeed', 'x-velocity','y-velocity'], axis=1)
event5=event5.rename({'Calculatedspeed': 'Speed'}, axis=1)
event6=event6.drop(['Unnamed:0','Interpolatedspeed', 'x-velocity','y-velocity'], axis=1)
event6=event6.rename({'Calculatedspeed': 'Speed'}, axis=1)
event1

In [ ]:
#prune data
if pruning==1:
    event1=prune(event1)
    event2=prune(event2)
    event3=prune(event3)
    event4=prune(event4)
    event5=prune(event5)
    event6=prune(event6)    
event1

In [ ]:
event1 = event1.drop(event1.columns[0:5], axis=1)
event2 = event2.drop(event2.columns[0:5], axis=1)
event3 = event3.drop(event3.columns[0:5], axis=1)
event4 = event4.drop(event4.columns[0:5], axis=1)
event5 = event5.drop(event5.columns[0:5], axis=1)
event6 = event6.drop(event6.columns[0:5], axis=1)
event1

In [ ]:
#count total cell actiations in each session
Session1 = event1[event1 > Spike_threshold].count(axis=0)
#Session2 = pd.Dataframe(Session1)
Session2 = event2[event2 > Spike_threshold].count(axis=0)
#Session3 = pd.Dataframe(Session1)
Session3 = event3[event3 > Spike_threshold].count(axis=0)
#Session4 = pd.Dataframe(Session1)
Session4 = event4[event4 > Spike_threshold].count(axis=0)
#Session5 = pd.Dataframe(Session1)
Session5 = event5[event5 > Spike_threshold].count(axis=0)
#Session6 = pd.Dataframe(Session1)
Session6 = event6[event6 > Spike_threshold].count(axis=0)

AcrossSessions = pd.concat([Session1, Session2, Session3, Session4, Session5, Session6], axis=1)
AcrossSessions = AcrossSessions.rename(columns={0: 'Session1', 1: 'Session2', 2: 'Session3', 3: 'Session4', 4: 'Session5', 5: 'Session6'})
AcrossSessions.index.name = 'ID'
AcrossSessions.reset_index(inplace=True)
AcrossSessions = pd.DataFrame(AcrossSessions)
AcrossSessions

## From here, check PC identification across 6 sessions

In [ ]:
#read in place cell identification data and combine
df_from_each_file = (pd.read_csv(f) for f in all_files)

concatenated_df   = pd.concat(df_from_each_file, axis=1, ignore_index=True) #dropping index as it was causing a problem to reasign the index...
concatenated_df

In [ ]:
#rearrange place cell identification data
df = concatenated_df.iloc[:, [0,3,7,11,15, 19, 23]]
df = df.set_axis(['cellID', 'PlaceCellS1', 'PlaceCellS2', 'PlaceCellS3', 'PlaceCellS4', 'PlaceCellS5', 'PlaceCellS6'], axis=1)
df

In [ ]:
#combine place cell identiciation data and cell activity count
df_all = pd.concat([df, AcrossSessions], axis=1)
df_all = df_all.drop(['ID'],axis=1)
df_all

In [ ]:
#write data to file
df_all.to_csv(f'{output_dir}/{testName}_eventcount&PC_raw(spike5).csv', index=True)

df_all